In [1]:
import os
%pwd

'f:\\ml_projects\\text_summarizer\\notbooks'

In [2]:
os.chdir("../")
%pwd

'f:\\ml_projects\\text_summarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir:Path
    transformed_data_path: Path
    model_ckpt: str
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [4]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILEPATH, params_filepath = PARAMS_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self):
        temp_config = self.config.model_trainer
        params = self.params.TrainingArguments
        create_directories([temp_config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = temp_config.root_dir,
            transformed_data_path = temp_config.transformed_data_path,
            model_ckpt = temp_config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [5]:
import os,sys
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset,load_from_disk
import torch
from text_summarizer.utils.exception import CustomException
from text_summarizer.utils.logger import logger

f:\ml_projects\text_summarizer\textsummarizer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 [ 2024-01-09 18:11:49,586 ] - 58 - config - PyTorch version 2.1.2 available.


In [6]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config

    def train(self):
        try:
            device = "cuda" if torch.cuda.is_available() else "cpu"
            tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
            model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
            seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model = model_pegasus)

            # Loading Dataset
            dataset_samsum_pt = load_from_disk(self.config.transformed_data_path)
            
            # training_args = TrainingArguments(
            #     output_dir = self.config.root_dir,
            #     num_train_epochs = self.config.num_train_epochs,
            #     warmup_steps = self.config.warmup_steps,
            #     per_device_train_batch_size = self.config.per_device_train_batch_size,
            #     per_device_eval_batch_size = self.config.per_device_train_batch_size,
            #     weight_decay = self.config.weight_decay,
            #     logging_steps = self.config.logging_steps,
            #     evaluation_strategy = self.config.evaluation_strategy,
            #     eval_steps = self.config.eval_steps,
            #     save_steps = self.config.save_steps,
            #     gradient_accumulation_steps = self.config.gradient_accumulation_steps
            # )
            
            training_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
           ) 

            trainer = Trainer(model = model_pegasus,
                            args = training_args,
                            tokenizer = tokenizer,
                            data_collator = seq2seq_data_collator,
                            train_dataset = dataset_samsum_pt['test'],
                            eval_dataset = dataset_samsum_pt['validation']
                            )
            
            trainer.train()
            logger.info("Training Has Completed")

            ## Save Model
            model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus_samsum_model"))

            ## Save Tokenizer
            tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))
            logger.info("Model and Tokenizer has been Saved...")
        except Exception as e:
            raise CustomException(e,sys)

In [7]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise CustomException(e,sys)

 [ 2024-01-09 18:11:51,758 ] - 25 - common - artifacts Directory Created Successfully
 [ 2024-01-09 18:11:51,766 ] - 25 - common - artifacts/model_trainer Directory Created Successfully


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/51 [00:00<?, ?it/s]You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 